In [24]:
import os
from dotenv import load_dotenv
from faker import Faker
import random
import time
import json
from confluent_kafka import Producer, Consumer

load_dotenv("F:\Pyspark-2025\pyspark\MongoDB\.env") # Load from .env file

fake = Faker()
conf = {
    'bootstrap.servers': os.getenv('bootstrap_server'),
    'security.protocol': 'SASL_SSL',
    'sasl.mechanism': 'PLAIN',
    'sasl.username': os.getenv('api_key2'),
    'sasl.password': os.getenv('api_secret2')
}


In [25]:
# Create a producer instance
producer = Producer(conf)

In [13]:
def generate_transactions():
    transaction = {
        "transaction_id": fake.uuid4(),
        "timestamp": int(time.time()),
        "user_id": random.randint(10000, 99999),
        "amount": round(random.uniform(5, 5000), 2),
        "transaction_type": random.choice(["purchase", "transfer", "withdrawal"]),
        "location": fake.city(),
        "merchant": fake.company(),
        "card_number": fake.credit_card_number()
    }
    return transaction

In [20]:
def delivery_report(err, msg):
    if err:
        print(f'Message delivery failed: {err}')
    else:
        print(f'Message delivered to {msg.topic()} [{msg.partition()}]')

In [ ]:
while True:
    txn = generate_transactions()
    producer.produce("txn_data",value = json.dumps(txn), key = str(txn['user_id']), callback=delivery_report)
    producer.flush()
